In [8]:
# Writes the coordinates of the center and the grain id to the output file

def write_center_id_inline_mem(x_coors, y_coors, z_coors, grain_ids, output_file, vis):
    x_dims = len(x_coors)
    y_dims = len(y_coors)
    z_dims = len(z_coors)
    index = 0
    x_pts = []
    y_pts = []
    z_pts = []

    with open(output_file, 'a') as f_append:
        for z_i in range(z_dims - 1):
            for y_i in range(y_dims - 1):
                for x_i in range(x_dims - 1):
                    temp = [0.0, 0.0, 0.0]
                    
                    # Find the middle of the voxel in all dimensions
                    temp[0] = (x_coors[x_i] + x_coors[x_i + 1]) / 2.0
                    temp[1] = (y_coors[y_i] + y_coors[y_i + 1]) / 2.0
                    temp[2] = (z_coors[z_i] + z_coors[z_i + 1]) / 2.0
                    
                    # Write the middle of the voxel and the grain id to the file
                    f_append.write(str(temp[0]) + ',')
                    f_append.write(str(temp[1]) + ',')
                    f_append.write(str(temp[2]) + ',')
                    f_append.write(str(grain_ids[index]) + '\n')
                    index += 1
                    if (vis):
                        x_pts.append(temp[0])
                        y_pts.append(temp[1])
                        z_pts.append(temp[2])
        
    if (vis):
        %pylab inline
        from matplotlib import pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
        from mpl_toolkits.mplot3d import proj3d

        fig = plt.figure() #figsize=(8,8)
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(x_pts, y_pts, z_pts,
                'o', markersize=1, color='blue', alpha=0.5, label='class1')
        plt.show()
    
# Read the dimensions from this line and return them as a list<int> of length 3

def read_dims(line):
    dims = [0,0,0]
    tokens = line.split(' ')
    dims[0] = int(tokens[1])
    dims[1] = int(tokens[2])
    dims[2] = int(tokens[3])
    return dims

# Read the coorindates from this line and return them as a list<float>

def read_coors(line):
    coors = []
    tokens = line.split(' ')
    for i in tokens:
        if (not '\n' in i):
            coors.append(float(i))
        elif i == '\n':
            pass
        else:
            coors.append(float(i.rstrip()))
    return coors

# Read the coorindates from this line and return them as a list<int>

def read_ids(line):
    grain_ids = []
    tokens = line.split(' ')
    for i in tokens:
        if (not '\n' in i):
            grain_ids.append(int(i))
        elif i == '\n':
            pass
        else:
            grain_ids.append(int(i.rstrip()))
    return grain_ids

def write_voxel_center_and_id(input_file, output_file, vis=False):
    
    # Lists of the coordinates of the voxel corners
    x_coors = []
    y_coors = []
    z_coors = []
    grain_id_index = 0

    # Set to true when reading coordinates of that dimension
    reading_x = False
    reading_y = False
    reading_z = False

    # Set to true when reading grain ids
    reading_id = False
        
    # Open the input file for reading
    with open(input_file, 'r') as file_read:
        
        # For each line in the input file
        for line in file_read:
            
            #1 Read the x, y, and z dimensions from this line
            if 'DIMENSIONS' in line:
                dims = read_dims(line)
                
            #3 If we're reading x's and haven't reached the y's yet
            elif (reading_x & (not 'Y_COORDINATES' in line)):
                # Append each coordinate in this line to x_coors
                for i in read_coors(line):
                    x_coors.append(i)
            
            #2 We're reading x's after this line
            elif 'X_COORDINATES' in line:
                reading_x = True
            
            #5
            elif (reading_y & (not 'Z_COORDINATES' in line)):
                for i in read_coors(line):
                    y_coors.append(i)
                    
            #4
            elif 'Y_COORDINATES' in line:
                reading_x = False
                reading_y = True
            
            #7 "CELL_DATA" is the marker that we're done with z's
            elif (reading_z & (not 'CELL_DATA' in line)):
                for i in read_coors(line):
                    z_coors.append(i)
            
            #6
            elif 'Z_COORDINATES' in line:
                reading_y = False
                reading_z = True
            
            #8 We're done reading z's when we hit "CELL DATA"
            elif 'CELL_DATA' in line:
                reading_z = False
            
            #10 For each grain id we read in this line, save it to grain_ids at the correct index
            elif reading_id:
                for grain_id in read_ids(line):
                    grain_ids[grain_id_index] = grain_id
                    grain_id_index += 1
            
            #9 We're done with coordinates. Write the file header and start writing grain ids
            elif 'LOOKUP_TABLE' in line:
                # Size of grain_ids is the number of cells
                grain_ids = [0] * (len(x_coors) - 1) * (len(y_coors) - 1) * (len(z_coors) - 1)
                
                # Write the file header to the output file
                with open(output_file, 'w') as f_write:
                    f_write.write('x coords,' + 'y coords,' + 'z coords,' + 'grain ids\n')
                reading_id = True
        
        # Append each x,y,z coordinate and grain id to the output file and print done when finished
        write_center_id_inline_mem(x_coors, y_coors, z_coors, grain_ids, output_file, vis)
        print('Done')

In [9]:
# Parameters are input and output paths

write_voxel_center_and_id('../data/0-D1_merged_all_cracks_2micron.vtk', '../data/1-voxel-centers.csv')

Percent Complete:  0.0033222591362126247
Percent Complete:  0.006644518272425249
Percent Complete:  0.009966777408637873
Percent Complete:  0.013289036544850499
Percent Complete:  0.016611295681063124
Percent Complete:  0.019933554817275746
Percent Complete:  0.023255813953488372
Percent Complete:  0.026578073089700997
Percent Complete:  0.029900332225913623
Percent Complete:  0.03322259136212625
Percent Complete:  0.036544850498338874
Percent Complete:  0.03986710963455149
Percent Complete:  0.04318936877076412
Percent Complete:  0.046511627906976744
Percent Complete:  0.04983388704318937
Percent Complete:  0.053156146179401995
Percent Complete:  0.05647840531561462
Percent Complete:  0.059800664451827246
Percent Complete:  0.06312292358803986
Percent Complete:  0.0664451827242525
Percent Complete:  0.06976744186046512
Percent Complete:  0.07308970099667775
Percent Complete:  0.07641196013289037
Percent Complete:  0.07973421926910298
Percent Complete:  0.08305647840531562
Percent Comp